In [98]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2
import os
from zipfile import ZipFile
import time
from datetime import datetime
import itertools

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras import utils
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

np.random.seed(42)
tf.random.set_seed(42)

In [99]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## DATASET

In [100]:
root_dir = 'E:\\Pycharm\\Age-Detection-Using-Neural-Network\\UTKFace_small'

# file csv lưu đường dẫn đến các ảnh trong dataset và nhãn của chúng
file_path = os.path.join(root_dir, 'DATA_FACE_Label.csv')

# đọc file csv
df = pd.read_csv(file_path)
df['path_name'] = df['image'].apply(lambda x: os.path.join(root_dir, x))
df['age'] = df['age'].apply(lambda x: 0 if x < 10 else (
    1 if x < 20 else (2 if x < 30 else (3 if x < 40 else (4 if x < 50 else (5 if x < 60 else (6 if x < 70 else 7)))))))

#chia thành 2 phần train và test tỉ lệ 80% train và 20% test

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [101]:
train_df['path_name'].head()


1404    E:\Pycharm\Age-Detection-Using-Neural-Network\...
721     E:\Pycharm\Age-Detection-Using-Neural-Network\...
518     E:\Pycharm\Age-Detection-Using-Neural-Network\...
199     E:\Pycharm\Age-Detection-Using-Neural-Network\...
1672    E:\Pycharm\Age-Detection-Using-Neural-Network\...
Name: path_name, dtype: object

In [102]:
train_df['age'].head()

1404    5
721     3
518     2
199     2
1672    7
Name: age, dtype: int64

In [103]:
train_filenames_list = list(train_df['path_name'])
train_labels_list = list(train_df['age'])

test_filenames_list = list(test_df['path_name'])
test_labels_list = list(test_df['age'])

In [104]:
len(train_filenames_list), len(train_labels_list), len(test_filenames_list), len(test_labels_list)

(1410, 1410, 353, 353)

In [105]:
num_classes =7


def _parse_function(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.io.decode_jpeg(image_string, channels=1)
    image_resized = tf.image.resize(image_decoded, [200, 200])  # Đúng kích thước input của model
    image_normalized = image_resized / 255.0
    label_onehot = tf.one_hot(label, num_classes)
    return image_normalized, label_onehot


train_dataset = tf.data.Dataset.from_tensor_slices((train_filenames_list, train_labels_list))
train_dataset = train_dataset.map(_parse_function)
train_dataset = train_dataset.batch(512).shuffle(1000)

test_dataset = tf.data.Dataset.from_tensor_slices((test_filenames_list, test_labels_list))
test_dataset = test_dataset.map(_parse_function)
test_dataset = test_dataset.batch(512)


## CNN Architecture

In [106]:

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=3, activation='relu',
                 input_shape=(200, 200, 1)))  # 3rd dim = 1 for grayscale images.
model.add(AveragePooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=256, kernel_size=3, activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))

model.add(GlobalAveragePooling2D())

model.add(Dense(132, activation='relu'))

model.add(Dense(7, activation='softmax'))

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 198, 198, 32)      320       
                                                                 
 average_pooling2d_28 (Avera  (None, 99, 99, 32)       0         
 gePooling2D)                                                    
                                                                 
 conv2d_29 (Conv2D)          (None, 97, 97, 64)        18496     
                                                                 
 average_pooling2d_29 (Avera  (None, 48, 48, 64)       0         
 gePooling2D)                                                    
                                                                 
 conv2d_30 (Conv2D)          (None, 46, 46, 128)       73856     
                                                                 
 average_pooling2d_30 (Avera  (None, 23, 23, 128)     

In [107]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Model fitting

In [108]:
model.fit(train_dataset, validation_data=test_dataset, epochs=10)


Epoch 1/10


ResourceExhaustedError: Graph execution error:

Detected at node 'sequential_7/conv2d_28/Relu' defined at (most recent call last):
    File "C:\Users\buian\anaconda3\envs\tf-gpu\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\buian\anaconda3\envs\tf-gpu\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\buian\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\buian\anaconda3\envs\tf-gpu\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\buian\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\buian\anaconda3\envs\tf-gpu\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\buian\anaconda3